# Segmenting and Clustering Neighborhoods in Toronto

_This notebook completed all of the Week 3 assignments in the Applied Data Science Capstone course on Coursera_

In [6]:
# Import all required libraries and install those not previously installed

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# !conda install -c conda-forge lxml --yes
import lxml

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
###
#
# First Submission
#
###

# Define the wikipedia URL
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Send a GET request to the wikipedia site
wiki_data = requests.get(wiki_url).text

# Get the first dataframe in the list, since it is the one of interest
wiki_contents = pd.read_html(wiki_data)[0]

# Drop all postal codes where a borough is not assigned and reassign column names
print("Number of postal codes imported: " + str(wiki_contents.shape[0]))
wiki_contents = wiki_contents[wiki_contents['Borough'] != 'Not assigned']
wiki_contents.columns = ['PostalCode', 'Borough', 'Neighborhood']
print("Number of postal codes with boroughs: " + str(wiki_contents.shape[0]))
wiki_contents.head()

Number of postal codes imported: 180
Number of postal codes with boroughs: 103


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
###
#
# Second Submission
#
###

# Create blank dataframe with columns
postalCodes = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

# Get data from provided csv file (geocoder wasn't working...)
latlng = pd.read_csv('Geospatial_Coordinates.csv')

# Iterate through all rows in the dataframe, getting the coordinates of each postal code
for index, row in wiki_contents.iterrows():
    
    # Get the row of the provided dataset matching the postal code
    coords = latlng[latlng['Postal Code'] == row['PostalCode']]
    
    # Append the results to the new dataframe
    postalCodes = postalCodes.append({'PostalCode': row['PostalCode'],
                                      'Borough': row['Borough'],
                                      'Neighborhood': row['Neighborhood'],
                                      'Latitude': coords.iloc[0]['Latitude'],
                                      'Longitude': coords.iloc[0]['Longitude']}, ignore_index=True)
postalCodes

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [4]:
# Function defined in previous labs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
###
#
# Third Submission
#
###

# Foursquare credentials
CLIENT_ID = '[...]'
CLIENT_SECRET = '[...]'
VERSION = '20180605'
LIMIT = 100

# Get center of Toronto coordinates to center map
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# Initialize Map
map_neighborhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# Send all neighborhoods to get nearby venues using provided function
toronto_venues = getNearbyVenues(names=postalCodes['Neighborhood'],
                                   latitudes=postalCodes['Latitude'],
                                   longitudes=postalCodes['Longitude']
                                  )

In [36]:
# Data cleaning = removal of categorical variables and grouping
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Generate Dataframe with top 10 venues in each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# Run k-Means clustering on the location data, clustering like neightborhoods
kclusters = 6
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = postalCodes

In [37]:
# Merge the venue data with the postal code data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [38]:
# Drop rows where the cluster label is NA
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]

In [39]:
# Create Map functions from other labs
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype('int32')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters